In [53]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [54]:
!pip install smart_open
from __future__ import division
import nltk 
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import *
from textblob.classifiers import NaiveBayesClassifier
from sklearn.model_selection import KFold
from nltk.classify.naivebayes import NaiveBayesClassifier
from gensim import corpora, models, similarities
from nltk.corpus import wordnet as wn
from smart_open import open

In [55]:
#Read dataset ISEAR.csv
Data = pd.read_csv('ISEAR.csv',header=None)

In [56]:
#Emotions to be detected
emotion_labels = ['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt']

In [57]:
#Negation words
negation_words = ['not', 'neither', 'nor', 'but', 'however', 'although', 'nonetheless', 'despite', 'except', 'even though', 'yet']

In [58]:
#Returns a list of all corresponding class labels
def class_labels(emotions):
    labels = []
    labelset = []
    exclude = []
    for i in range(len(emotions)):
#         labels.append(e)
#         labelset.append([e])
        if emotions[i] not in ['shame','guilt']:
            labels.append(e)
            labelset.append([e])
        else:
            exclude.append(i)
    return labels, labelset, exclude

In [59]:
#Removes unnecessary characters from sentences
def removal(sentences):
    sentence_list = []
    count = 0
#     for sen in sentences:
#         count += 1
#         print count
#         print sen
#         print type(sen)
    s = nltk.word_tokenize(sentences)
    characters = ["á", "\xc3", "\xa1", "\n", ",", ".", "[", "]", ""]
    l = []
    for t in s:
        if t not in characters:
            l.append(t)
    return l
#     new = ' '.join([i for i in s if not [e for e in characters if e in i]])
#     print new
#     sentence_list.append(new)
#     return sentence_list

In [60]:
#POS-TAGGER, returns NAVA words
def pos_tag(sentences):
    tags = [] #have the pos tag included
    nava_sen = []
    pt = nltk.pos_tag(sentences)
#     for s in sentences:
#     s_token = nltk.word_tokenize(sentences)
#     pt = nltk.pos_tag(s_token)
    nava = []
    nava_words = []
    for t in pt:
        if t[1].startswith('NN') or t[1].startswith('JJ') or t[1].startswith('VB') or t[1].startswith('RB'):
            nava.append(t)
            nava_words.append(t[0])
    return nava, nava_words
#     tags.append(nava)
#     nava_sen.append(nava_words)
#     return tags, nava_sen

In [61]:
#Performs stemming
def stemming(sentences):
    sentence_list = []
    sen_string = []
    sen_token = []
    stemmer = PorterStemmer()
    i = 0
#     for sen in sentences:
#         print i,
    i += 1
    st = ""
    for word in sentences:
        word_l = word.lower()
        if len(word_l) >= 3:
            st += stemmer.stem(word_l) + " "
    sen_string.append(st)
    w_set = nltk.word_tokenize(st)
    sen_token.append(w_set)
    w_text = nltk.Text(w_set)
    sentence_list.append(w_text)
    return w_text, st, w_set
#     return sentence_list, sen_string, sen_token

In [62]:
#Write to file
def write_to_file(filename, text):
    o = open(filename,'w')
    o.write(str(text))
    o.close()

In [63]:
#Creating the dataframe
def create_frame(Data):
    labels = []
#     sentences = []
#     sen_string = []
#     sen_token =[]
    sen = []
    sen_s = []
    sen_t = []
    labelset = []
    for i in range(len(Data)):
        if i >= 0:
#             print i,
            emotion = Data[0][i]
            sit = Data[1][i]
#             if emotion not in ['shame', 'guilt']:
            labels.append(emotion)
            labelset.append([emotion])
            sent = removal(sit)
            nava, sent_pt = pos_tag(sent)
            sentences, sen_string, sen_token = stemming(sent_pt)
            sen.append(sentences)
            sen_s.append(sen_string)
            sen_t.append(sen_token)
#     labels, labelset, exclude = class_labels(emotions[1:])
#     sent = removal(sit[1:], exclude)
#     nava, sent_pt = pos_tag(sent)
#     sentences, sen_string, sen_token = stemming(sent_pt)
    frame = pd.DataFrame({0 : labels,
                          1 : sen,
                          2 : sen_s,
                          3 : sen_t,
                          4 : labelset})
    return frame, sen_t, labels, sen_s

In [64]:
#Calling the create_frame function
c, st, labels, senten = create_frame(Data)

In [65]:
#Display of frames with classification of words on emotions labels
c

,0,1,2,3,4
0,joy,"(day, feel, close, partner, other, friend, fee...",day feel close partner other friend feel peac ...,"[day, feel, close, partner, other, friend, fee...",[joy]
1,fear,"(time, imagin, someon, love, contact, seriou, ...",time imagin someon love contact seriou ill eve...,"[time, imagin, someon, love, contact, seriou, ...",[fear]
2,anger,"(had, been, obvious, unjustli, treat, had, pos...",had been obvious unjustli treat had possibl el...,"[had, been, obvious, unjustli, treat, had, pos...",[anger]
3,sadness,"(think, short, time, live, relat, period, life...",think short time live relat period life think ...,"[think, short, time, live, relat, period, life...",[sadness]
4,disgust,"(gather, found, involuntarili, sit, next, peop...",gather found involuntarili sit next peopl expr...,"[gather, found, involuntarili, sit, next, peop...",[disgust]
...,...,...,...,...,...
7511,shame,"(year, back, someon, invit, tutor, grand-daugh...",year back someon invit tutor grand-daught gran...,"[year, back, someon, invit, tutor, grand-daugh...",[shame]
7512,shame,"(had, taken, respons, someth, had, prepar, how...",had taken respons someth had prepar howev fail...,"[had, taken, respons, someth, had, prepar, how...",[shame]
7513,fear,"(wa, home, heard, loud, sound, spit, door, tho...",wa home heard loud sound spit door thought fam...,"[wa, home, heard, loud, sound, spit, door, tho...",[fear]
7514,guilt,"(did, not, homework, teacher, had, ask, wa, sc...",did not homework teacher had ask wa scold immedi,"[did, not, homework, teacher, had, ask, wa, sc...",[guilt]


In [66]:
#Reads the emotion representative words file
def readfile(filename):
    f = open(filename,'r')
    representative_words = []
    for line in f.readlines():
        characters = ["\n", " ", "\r", "\t"]
        new = ''.join([i for i in line if not [e for e in characters if e in i]])
        representative_words.append(new)
    return representative_words

In [67]:
#Makes a list of all words semantically related to an emotion and Stemming
def affect_wordlist(words):
    affect_words = []
    stemmer = PorterStemmer()
    for w in words:
        w_l = w.lower()
        word_stem = stemmer.stem(w_l)
        if word_stem not in affect_words:
            affect_words.append(word_stem)
    return affect_words

In [68]:
#Creating an emotion wordnet
def emotion_word_set(emotions):
    word_set = {}
    for e in emotions:
        representative_words = readfile(e)
        wordlist = affect_wordlist(representative_words)
        word_set[e] = wordlist
    return word_set

In [69]:
#Emotion Detector - Getting synonyms from wordnet synsets
def get_synonyms():
    syn = {}
    for e in emotion_labels:
        jw = wn.synsets(e)
        for s in jw:
            v = s.name()
            try:
                syn[e].append(wn.synset(v).lemma_names())
            except KeyError:
                syn[e] = wn.synset(v).lemma_names()

In [70]:
#Emotion Detector - Creating training/testing set for Naive Bayes classifier TextBlob -- Not used
def create_dataset_textblob(sentences, emotions):
    train = []
    sen = []
    emo = []
    for s in sentences:
        sen.append(s)
    for e in emotions:
        emo.append(e)
    for i in range(len(sen)):
        s = sen[i]
        e = emo[i]
        train.append((str(s), e))
    return train

In [71]:
#Testing for Naive Bayes Classifier
def testing(cl, test):
    for s, e in test:
        r = cl.classify(s)
        print(s, e, r)
        if r == e:
            print("*")

In [72]:
#Create dataset for nltk Naive Bayes
def create_data(sentence, emotion):
    data = []
    for i in range(len(sentence)):
        sen = []
        for s in sentence[i]:
            sen.append(str(s))
        emo = emotion[i]
        data.append((sen, emo))
    return data

In [73]:
#Get all words in dataset
def get_words_in_dataset(dataset):
    all_words = []
    for (words, sentiment) in dataset:
        all_words.extend(words)
    return all_words

In [74]:
#Getting frequency dist of words
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

In [75]:
#Extacting features
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [76]:
#Create test data
def create_test(sentence, emotion):
    data = []
    sen = []
    emo = []
    for s in sentence:
        sen.append(str(s))
    for e in emotion:
        emo.append(e)
    for i in range(len(sen)):
        temp = []
        temp.append(sen[i])
        temp.append(emo[i])
        data.append(temp)
    return data

In [78]:
#Classifier
def classify_dataset(data):
    return \
        classifier.classify(extract_features(nltk.word_tokenize(data)))

In [80]:
#Get accuracy
def get_accuracy(test_data, classifier):
    total = accuracy = float(len(test_data))
    for data in test_data:
        if classify_dataset(data[0]) != data[1]:
#             print data, classify_dataset(data[0]), data[1]
            accuracy -= 1
    print('Total accuracy: %f%% (%d/20).' % (accuracy / total * 100, accuracy))
    final = accuracy / total * 100
    return final

In [81]:
# Create training and testing data
sen = c[3]
emo = c[0]
l = len(c[3])
limit = (9*l)//10
sente = c[2]
Data = create_data(sen[:limit], emo[:limit])
test_data = create_test(sente[limit:], emo[limit:])

In [82]:
# extract the word features out from the training data
word_features = get_word_features(\
                    get_words_in_dataset(Data))

In [83]:
# get the training set and train the Naive Bayes Classifier
training_set = nltk.classify.util.apply_features(extract_features, Data)
classifier = NaiveBayesClassifier.train(training_set)

In [84]:
Naive_accu = get_accuracy(test_data, classifier)

Total accuracy: 63.164894% (475/20).


In [85]:
print("Accuracy using Naive Bayes Component  ", Naive_accu, "%")

Accuracy using Naive Bayes Component   63.16489361702128 %
